## LangChain chains

In [5]:
!pip install -q langchain langchain-openai python-dotenv

### Prosty chain: prompt → model → wynik

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_messages([
    ("system", "jesteś ekspertem programowania w Pythonie i geniuszem w dziedzinie AI."),
    ("user", "napisz kod {topic}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()
result = chain.invoke({'topic': 'AGI'})


In [10]:
print(result)

Tworzenie ogólnej inteligencji sztucznej (AGI) to niezwykle złożone i ambitne zadanie, które wykracza poza możliwości pojedynczego programu czy fragmentu kodu. AGI odnosi się do systemu, który potrafi uczyć się, rozumieć i stosować wiedzę w różnych dziedzinach, podobnie jak człowiek. Obecnie nie istnieje żaden powszechnie akceptowany model AGI, a badania w tej dziedzinie są wciąż w toku.

Jednak mogę zaproponować prosty przykład kodu w Pythonie, który ilustruje podstawowe elementy uczenia maszynowego, które są częścią większego obrazu AGI. Poniżej znajduje się przykład prostego modelu uczenia maszynowego przy użyciu biblioteki `scikit-learn`, który klasyfikuje dane.

```python
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Wczytanie zbioru danych (np. Iris)
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Podział danych na z

### Sequential chain: dwa modele w sekwencji

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Chain: streszczenie (wejście: {tekst} → wyjście: str)
summary_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Streść poniższy tekst w 1–2 zdaniach po polsku."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

# 2) Adapter: zamień str → {"tekst": str}, żeby podać do następnego promptu
to_dict = RunnableLambda(lambda s: {"tekst": s})

# 3) Chain: tłumaczenie streszczenia na francuski (wejście: {tekst} → wyjście: str)
translate_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Przetłumacz tekst na język francuski."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

sequential_chain = summary_chain | to_dict | translate_chain
input_text = "LangChain umożliwia tworzenie aplikacji AI poprzez łączenie modeli, promptów i narzędzi w spójne pipeline’y."

final_translation = sequential_chain.invoke({"tekst": input_text})

print(final_translation)


LangChain permet de créer des applications d'IA en intégrant des modèles, des invites et des outils dans des processus organisés.


### Branching chain: jedna odpowiedź, dwa przetworzenia

In [12]:
from langchain_core.runnables import RunnableParallel

# Prompt do streszczenia
prompt_summary = ChatPromptTemplate.from_template("Streść: {tekst}")

# Prompt do sentymentu
prompt_sentiment = ChatPromptTemplate.from_template("Określ ton wypowiedzi: {tekst}")

branch_chain = RunnableParallel(
    summary=(prompt_summary | llm | StrOutputParser()),
    sentiment=(prompt_sentiment | llm | StrOutputParser())
)

text = "Jestem bardzo zadowolony z tego kursu, nauczyłem się dużo o LangChain!"
result = branch_chain.invoke({"tekst": text})

print(result)


{'summary': 'Uczestnik kursu wyraża swoje zadowolenie, podkreślając, że zdobył wiele wiedzy na temat LangChain.', 'sentiment': 'Ton wypowiedzi jest pozytywny i entuzjastyczny. Osoba wyraża zadowolenie z kursu oraz podkreśla, że zdobyła cenną wiedzę na temat LangChain.'}
